# mumbai

In [7]:
from hashlib import new
from tokenize import blank_re
import pygsheets #Importing python in google sheets
import pandas as pd #importing pandas
from pandas import Series,DataFrame
import datetime
from datetime import datetime, timedelta,date
import numpy as np
import datetime as dt





# gc = pygsheets.authorize(service_file='/Users/karan_19981/Desktop/Work/Everest_Fleet/Database_Sheets/client_secret.json')

gc = pygsheets.authorize(service_file='client_secret.json')


year = int(input('Enter a year:'))
month = int(input('Enter a month:'))
day = int(input('Enter a day:'))



last_week_day = datetime(year, month, day)

print(last_week_day)

today = date.today()




next_day = last_week_day + timedelta(days = 1)

print(next_day)

car_master_sheet_mumbai = gc.open("Car Master Sheet")
cms_mumbai_recovery_tab = car_master_sheet_mumbai.worksheet_by_title("Recovery")
cms_mumbai_recovery_data = pd.DataFrame(cms_mumbai_recovery_tab.get_all_records())
cms_mumbai_recovery_data = cms_mumbai_recovery_data[['Date','ETMID','Amount','DM Name']]
cms_mumbai_recovery_data.rename(columns={"ETMID":"ETM",'DM Name':"Remarks"},inplace = True)
cms_mumbai_recovery_data['Type'] = 'RECOVERY'
print(cms_mumbai_recovery_data.head())
cms_mumbai_recovery_data = cms_mumbai_recovery_data[['Date','ETM','Amount','Remarks','Type']]
print(cms_mumbai_recovery_data.head())

cms_mumbai_penalty_tab = car_master_sheet_mumbai.worksheet_by_title("Penalty")
cms_mumbai_penalty_data = pd.DataFrame(cms_mumbai_penalty_tab.get_all_records())
cms_mumbai_penalty_data = cms_mumbai_penalty_data[['Date in Payout','ETM','Penalty','Fine Amount']]
cms_mumbai_penalty_data.rename(columns={"Date in Payout":"Date",'Penalty':"Remarks","Fine Amount":"Amount"},inplace = True)
cms_mumbai_penalty_data['Type'] = np.where((cms_mumbai_penalty_data['Remarks'] == "Without Intimation"), "UNSCHEDULED_LEAVES", "ACCIDENT")
print(cms_mumbai_penalty_data.head())

cms_mumbai_B2B_tab = car_master_sheet_mumbai.worksheet_by_title("B2B")
cms_mumbai_B2B_data = pd.DataFrame(cms_mumbai_B2B_tab.get_all_records())
cms_mumbai_B2B_data = cms_mumbai_B2B_data[['Date','ETMID','Duty','TotalDutyPayout','Fuel','Toll']]
cms_mumbai_B2B_data.rename(columns={"ETMID":"ETM",'Duty':"Remarks"},inplace = True)

cms_mumbai_B2B_duty = cms_mumbai_B2B_data[['Date','ETM','Remarks','TotalDutyPayout']]
cms_mumbai_B2B_duty['Type'] = 'B2B_DUTY'
cms_mumbai_B2B_duty.rename(columns={"TotalDutyPayout":"Amount"},inplace = True)

cms_mumbai_B2B_fuel = cms_mumbai_B2B_data[['Date','ETM','Remarks','Fuel']]
cms_mumbai_B2B_fuel['Type'] = 'B2B_FUEL'
cms_mumbai_B2B_fuel.rename(columns={"Fuel":"Amount"},inplace = True)


cms_mumbai_B2B_toll = cms_mumbai_B2B_data[['Date','ETM','Remarks','Toll']]
cms_mumbai_B2B_toll['Type'] = 'B2B_TOLL'
cms_mumbai_B2B_toll.rename(columns={"Toll":"Amount"},inplace = True)



cms_mumbai_rto_tab = car_master_sheet_mumbai.worksheet_by_title("RTO")
cms_mumbai_rto_data = pd.DataFrame(cms_mumbai_rto_tab.get_all_records())
cms_mumbai_rto_data = cms_mumbai_rto_data[['Date in Payout','ETM','Penalty','Fine Amount']]
cms_mumbai_rto_data.rename(columns={"Date in Payout":"Date",'Penalty':"Remarks","Fine Amount":"Amount"},inplace = True)
cms_mumbai_rto_data['Type'] = 'RTO_FINE'





cms_mumbai_adjustment_tab = car_master_sheet_mumbai.worksheet_by_title("Adjustments")
cms_mumbai_adjustment_data = pd.DataFrame(cms_mumbai_adjustment_tab.get_all_records())
cms_mumbai_adjustment_data = cms_mumbai_adjustment_data[['Date','ETM ID','Type','Remark','Amount']]
cms_mumbai_adjustment_data.rename(columns={"ETM ID":"ETM",'Remark':"Remarks","Type":"Test Type"},inplace = True)


cms_mumbai_adjustment_data['Amount'] = cms_mumbai_adjustment_data['Amount'].replace(',','', regex=True)



cms_mumbai_adjustment_data[['Amount']] = cms_mumbai_adjustment_data[['Amount']].apply(pd.to_numeric)

col         = 'Test Type'
col1 = 'Amount'
conditions  = [ cms_mumbai_adjustment_data[col].str.contains("fuel",case = False),cms_mumbai_adjustment_data[col].str.contains("refer",case = False), cms_mumbai_adjustment_data[col].str.contains("life",case = False), cms_mumbai_adjustment_data[col].str.contains("repair",case = False),cms_mumbai_adjustment_data[col].str.contains("pun",case = False),cms_mumbai_adjustment_data[col].str.contains("toll",case = False),cms_mumbai_adjustment_data[col].str.contains("Penalty",case = False),cms_mumbai_adjustment_data[col].str.contains("reversal",case = False),cms_mumbai_adjustment_data[col].str.contains("joining fee",case = False),cms_mumbai_adjustment_data[col1] >= 0,cms_mumbai_adjustment_data[col1] < 0 ]
choices     = [ "FUEL_ADJUSTMENT","DRIVER_REFERENCE", 'LIFETIME_INCENTIVE', 'REPAIRS','REPAIRS','TOLL','PENALTY_REVERSAL','PENALTY_REVERSAL','JOINING_FEE','OTHER_ADDITIONS','OTHER_DEDUCTIONS']
    
cms_mumbai_adjustment_data["Type"] = np.select(conditions, choices, default=np.nan)

cms_mumbai_adjustment_data = cms_mumbai_adjustment_data[['Date','ETM','Remarks','Amount','Type']]





cms_mumbai_amount_paid_tab = car_master_sheet_mumbai.worksheet_by_title("Amount Paid")
cms_mumbai_amount_paid_data = pd.DataFrame(cms_mumbai_amount_paid_tab.get_all_records())
cms_mumbai_amount_paid_data = cms_mumbai_amount_paid_data[['Date','ETM','Remarks','Amount']]
cms_mumbai_amount_paid_data['Type'] = 'BANK_TRANSFER'


final_mumbai_data = pd.concat([cms_mumbai_recovery_data, cms_mumbai_amount_paid_data,cms_mumbai_adjustment_data,cms_mumbai_rto_data,cms_mumbai_B2B_toll,cms_mumbai_B2B_fuel,cms_mumbai_B2B_duty,cms_mumbai_penalty_data],ignore_index = True)

final_mumbai_data['Date'] = pd.to_datetime(final_mumbai_data['Date'],dayfirst=True)


final_mumbai_data['start date'] = last_week_day
final_mumbai_data['end date'] = next_day

final_mumbai_data = final_mumbai_data.loc[(final_mumbai_data['start date'] <= final_mumbai_data['Date']) & (final_mumbai_data['end date'] > final_mumbai_data['Date'])]

final_mumbai_data['ETM_NEW'] = final_mumbai_data['ETM'].str.upper() 


final_mumbai_data = final_mumbai_data[['ETM_NEW','Amount','Type','Remarks','Date']]

final_mumbai_data.rename(columns={"ETM_NEW":"ETM"},inplace = True)

final_mumbai_data_first =  final_mumbai_data.groupby(['ETM','Type'],as_index = False).first()

final_mumbai_data_first.drop(['Amount'],axis = 1, inplace = True)

final_mumbai_data = final_mumbai_data[['ETM','Amount','Type']]

final_mumbai_data['Amount'] = final_mumbai_data['Amount'].replace(',','', regex=True)



final_mumbai_data[['Amount']] = final_mumbai_data[['Amount']].apply(pd.to_numeric)



final_mumbai_data_sum =  final_mumbai_data.groupby(['ETM','Type'],as_index = False).sum()


final_mumbai_data= final_mumbai_data_sum.merge(final_mumbai_data_first, on=["ETM","Type"])

final_mumbai_data = final_mumbai_data.loc[final_mumbai_data['Amount'] != 0]

final_mumbai_data = final_mumbai_data[['Date','ETM','Type','Amount','Remarks']]



cms_test = gc.open("testing")
final_result = cms_test.worksheet_by_title("cms")


final_result.set_dataframe(final_mumbai_data,'A1')

Enter a year:2022
Enter a month:07
Enter a day:01
2022-07-01 00:00:00
2022-07-02 00:00:00
         Date ETM Amount    Remarks      Type
0  04/01/2021       2500  Suspenses  RECOVERY
1  04/01/2021       2027  Suspenses  RECOVERY
2  04/01/2021        463  Suspenses  RECOVERY
3  04/01/2021        500  Suspenses  RECOVERY
4  04/01/2021       1077  Suspenses  RECOVERY
         Date ETM Amount    Remarks      Type
0  04/01/2021       2500  Suspenses  RECOVERY
1  04/01/2021       2027  Suspenses  RECOVERY
2  04/01/2021        463  Suspenses  RECOVERY
3  04/01/2021        500  Suspenses  RECOVERY
4  04/01/2021       1077  Suspenses  RECOVERY
         Date       ETM                       Remarks  Amount      Type
0  27/12/2021  ETM13572             Tail Light Damage -1185.0  ACCIDENT
1  27/12/2021  Etm12944  Side Rear View Mirror Broken  -850.0  ACCIDENT
2  28/12/2021  ETM13642                      Accident -4000.0  ACCIDENT
3  28/12/2021  ETM10547                      Accident -2000.0  ACCIDEN

C:\Users\sagar\AppData\Local\Temp/ipykernel_18812/1811687309.py:62: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cms_mumbai_B2B_duty['Type'] = 'B2B_DUTY'
C:\Users\sagar\anaconda3\lib\site-packages\pandas\core\frame.py:5039: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(
C:\Users\sagar\AppData\Local\Temp/ipykernel_18812/1811687309.py:66: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydat

In [13]:
car_master_sheet_mumbai = gc.open("Car Master Sheet")
cms_mumbai_rto_tab = car_master_sheet_mumbai.worksheet_by_title("RTO")
cms_mumbai_rto_data = pd.DataFrame(cms_mumbai_rto_tab.get_all_records())
cms_mumbai_rto_data = cms_mumbai_rto_data[['Date in Payout','ETM','Penalty','Fine Amount']]
cms_mumbai_rto_data.rename(columns={"Date in Payout":"Date",'Penalty':"Remarks","Fine Amount":"Amount"},inplace = True)
cms_mumbai_rto_data['Type'] = 'RTO_FINE'
cms_mumbai_rto_data

,Date,ETM,Remarks,Amount,Type
0,01-01-2022,ETM13520,Without Prescribed Uniform (Tourist Cab),-500,RTO_FINE
1,01-01-2022,ETM08761,Jumping signal,-500,RTO_FINE
2,01-01-2022,ETM06251,Jumping signal,-500,RTO_FINE
3,01-01-2022,ETM06251,Without Prescribed Uniform (Tourist Cab),-500,RTO_FINE
4,01-01-2022,ETM13487,Speed violating by driver (LMV),-2000,RTO_FINE
...,...,...,...,...,...
5276,23/07/2022,ETM23321,Causing danger/Obstructions/Undue inconvenienc...,-500,RTO_FINE
5277,23/07/2022,ETM23337,"Parking the vehicle in ""No Parking"" area",-1,RTO_FINE
5278,23/07/2022,ETM10105,Causing danger/Obstructions/Undue inconvenienc...,-500,RTO_FINE
5279,23/07/2022,ETM19938,Prohibition on unnecessary use of horn/no honking,-1000,RTO_FINE


# pune

In [47]:
#importing libraries

from hashlib import new
from tokenize import blank_re
import pygsheets #Importing python in google sheets
import pandas as pd #importing pandas
from pandas import Series,DataFrame
import datetime
from datetime import datetime, timedelta,date
import numpy as np
import datetime as dt



gc = pygsheets.authorize(service_file='client_secret.json')


year = int(input('Enter a year:'))
month = int(input('Enter a month:'))
day = int(input('Enter a day:'))



last_week_day = datetime(year, month, day)

print(last_week_day)

today = date.today()




next_day = last_week_day + timedelta(days = 1)

print(next_day)


#recovery tab

car_master_sheet_pune = gc.open("Pune Car Master Sheet")
cms_pune_recovery_tab = car_master_sheet_pune.worksheet_by_title("Recovery")
cms_pune_recovery_data = pd.DataFrame(cms_pune_recovery_tab.get_all_records())
cms_pune_recovery_data = cms_pune_recovery_data[['Date','ETM','Amount','DM Name']]
cms_pune_recovery_data.rename(columns={'DM Name':"Remarks"},inplace = True)
cms_pune_recovery_data['Type'] = 'RECOVERY'
print(cms_pune_recovery_data.head())
cms_pune_recovery_data = cms_pune_recovery_data[['Date','ETM','Amount','Remarks','Type']]
print(cms_pune_recovery_data.head())


#penalty tab

cms_pune_penalty_tab = car_master_sheet_pune.worksheet_by_title("Penalty")
cms_pune_penalty_data = pd.DataFrame(cms_pune_penalty_tab.get_all_records())
cms_pune_penalty_data = cms_pune_penalty_data[['Date in Payout','ETM','Penalty','Fine Amount']]
cms_pune_penalty_data.rename(columns={"Date in Payout":"Date",'Penalty':"Remarks","Fine Amount":"Amount"},inplace = True)
cms_pune_penalty_data['Type'] = np.where((cms_pune_penalty_data['Remarks'] == "Without Intimation"), "UNSCHEDULED_LEAVES", "ACCIDENT")
print(cms_pune_penalty_data.head())

#b2b tab

# cms_mumbai_B2B_tab = car_master_sheet_mumbai.worksheet_by_title("B2B")
# cms_mumbai_B2B_data = pd.DataFrame(cms_mumbai_B2B_tab.get_all_records())
# cms_mumbai_B2B_data = cms_mumbai_B2B_data[['Date','ETMID','Duty','TotalDutyPayout','Fuel','Toll']]
# cms_mumbai_B2B_data.rename(columns={"ETMID":"ETM",'Duty':"Remarks"},inplace = True)

# cms_mumbai_B2B_duty = cms_mumbai_B2B_data[['Date','ETM','Remarks','TotalDutyPayout']]
# cms_mumbai_B2B_duty['Type'] = 'B2B_DUTY'
# cms_mumbai_B2B_duty.rename(columns={"TotalDutyPayout":"Amount"},inplace = True)

# cms_mumbai_B2B_fuel = cms_mumbai_B2B_data[['Date','ETM','Remarks','Fuel']]
# cms_mumbai_B2B_fuel['Type'] = 'B2B_FUEL'
# cms_mumbai_B2B_fuel.rename(columns={"Fuel":"Amount"},inplace = True)


# cms_mumbai_B2B_toll = cms_mumbai_B2B_data[['Date','ETM','Remarks','Toll']]
# cms_mumbai_B2B_toll['Type'] = 'B2B_TOLL'
# cms_mumbai_B2B_toll.rename(columns={"Toll":"Amount"},inplace = True)

#rto tab

cms_pune_rto_tab = car_master_sheet_pune.worksheet_by_title("RTO")
cms_pune_rto_data = pd.DataFrame(cms_pune_rto_tab.get_all_records())
cms_pune_rto_data = cms_pune_rto_data[['Date in Payout','ETM','Penalty','Fine Amount']]
cms_pune_rto_data.rename(columns={"Date in Payout":"Date",'Penalty':"Remarks","Fine Amount":"Amount"},inplace = True)
cms_pune_rto_data['Type'] = 'RTO_FINE'
print(cms_pune_rto_data.head())

#adjustment

cms_pune_adjustment_tab = car_master_sheet_pune.worksheet_by_title("Adjustments")
cms_pune_adjustment_data = pd.DataFrame(cms_pune_adjustment_tab.get_all_records())
cms_pune_adjustment_data = cms_pune_adjustment_data[['Date','ETM ID','Type','Remark','Amount']]
cms_pune_adjustment_data.rename(columns={"ETM ID":"ETM",'Remark':"Remarks","Type":"Test Type"},inplace = True)

cms_pune_adjustment_data['Amount'] = cms_pune_adjustment_data['Amount'].replace(',','', regex=True)

cms_pune_adjustment_data[['Amount']] = cms_pune_adjustment_data[['Amount']].apply(pd.to_numeric)

col         = 'Test Type'
col1 = 'Amount'
conditions  = [ cms_pune_adjustment_data[col].str.contains("fuel",case = False),cms_pune_adjustment_data[col].str.contains("refer",case = False), cms_pune_adjustment_data[col].str.contains("life",case = False), cms_pune_adjustment_data[col].str.contains("repair",case = False),cms_pune_adjustment_data[col].str.contains("pun",case = False),cms_pune_adjustment_data[col].str.contains("toll",case = False),cms_pune_adjustment_data[col].str.contains("Penalty",case = False),cms_pune_adjustment_data[col].str.contains("reversal",case = False),cms_pune_adjustment_data[col].str.contains("joining fee",case = False),cms_pune_adjustment_data[col1] >= 0,cms_pune_adjustment_data[col1] < 0 ]
choices     = [ "FUEL_ADJUSTMENT","DRIVER_REFERENCE", 'LIFETIME_INCENTIVE', 'REPAIRS','REPAIRS','TOLL','PENALTY_REVERSAL','PENALTY_REVERSAL','JOINING_FEE','OTHER_ADDITIONS','OTHER_DEDUCTIONS']
    
cms_pune_adjustment_data["Type"] = np.select(conditions, choices, default=np.nan)

cms_pune_adjustment_data = cms_pune_adjustment_data[['Date','ETM','Remarks','Amount','Type']]
print(cms_pune_adjustment_data.head())

#amount paid

cms_pune_amount_paid_tab = car_master_sheet_pune.worksheet_by_title("Amount Paid")
cms_pune_amount_paid_data = pd.DataFrame(cms_pune_amount_paid_tab.get_all_records())
cms_pune_amount_paid_data = cms_pune_amount_paid_data[['Date','ETM','Remarks','Amount']]
cms_pune_amount_paid_data['Type'] = 'BANK_TRANSFER'
print(cms_pune_amount_paid_data.head())


#concating 

final_pune_data = pd.concat([cms_pune_recovery_data, cms_pune_amount_paid_data,cms_pune_adjustment_data,cms_pune_rto_data,cms_pune_penalty_data],ignore_index = True)
final_pune_data['Date'] = pd.to_datetime(final_pune_data['Date'],dayfirst=True)
final_pune_data['start date'] = last_week_day
final_pune_data['end date'] = next_day
final_pune_data = final_pune_data.loc[(final_pune_data['start date'] <= final_pune_data['Date']) & (final_pune_data['end date'] > final_pune_data['Date'])]
final_pune_data['ETM_NEW'] = final_pune_data['ETM'].str.upper() 
print(final_pune_data.head())


final_pune_data = final_pune_data[['ETM_NEW','Amount','Type','Remarks','Date']]
final_pune_data.rename(columns={"ETM_NEW":"ETM"},inplace = True)
final_pune_data_first =  final_pune_data.groupby(['ETM','Type'],as_index = False).first()
final_pune_data_first.drop(['Amount'],axis = 1, inplace = True)
final_pune_data = final_pune_data[['ETM','Amount','Type']]
final_pune_data['Amount'] = final_pune_data['Amount'].replace(',','', regex=True)
final_pune_data[['Amount']] = final_pune_data[['Amount']].apply(pd.to_numeric)
print(final_pune_data.head())

final_pune_data_sum =  final_pune_data.groupby(['ETM','Type'],as_index = False).sum()
final_pune_data= final_pune_data_sum.merge(final_pune_data_first, on=["ETM","Type"])
final_pune_data = final_pune_data.loc[final_pune_data['Amount'] != 0]
final_pune_data = final_pune_data[['Date','ETM','Type','Amount','Remarks']]
print(final_pune_data.head())

#pushing the final result

cms_test = gc.open("testing")
final_result = cms_test.worksheet_by_title("cms_pune")
final_result.set_dataframe(final_pune_data,'A1')


Enter a year:2022
Enter a month:07
Enter a day:01
2022-07-01 00:00:00
2022-07-02 00:00:00
         Date       ETM  Amount          Remarks      Type
0  09-02-2022  ETM04087    1271   Silent Killers  RECOVERY
1  22-02-2022  ETM04087    2757   Silent Killers  RECOVERY
2  01-03-2022  ETM04087     762   Silent Killers  RECOVERY
3  08-03-2022  ETM17441    1186  Terrific Tigers  RECOVERY
4  08-03-2022  ETM04087    2811   Silent Killers  RECOVERY
         Date       ETM  Amount          Remarks      Type
0  09-02-2022  ETM04087    1271   Silent Killers  RECOVERY
1  22-02-2022  ETM04087    2757   Silent Killers  RECOVERY
2  01-03-2022  ETM04087     762   Silent Killers  RECOVERY
3  08-03-2022  ETM17441    1186  Terrific Tigers  RECOVERY
4  08-03-2022  ETM04087    2811   Silent Killers  RECOVERY
         Date       ETM                         Remarks  Amount      Type
0  14/03/2022  ETM17441         Winshield Mirror Broken    -850  ACCIDENT
1  08-06-2022  ETM20518                    Car Acciden

# vinay

In [45]:
#importing libraries

from hashlib import new
from tokenize import blank_re
import pygsheets #Importing python in google sheets
import pandas as pd #importing pandas
from pandas import Series,DataFrame
import datetime
from datetime import datetime, timedelta,date
import numpy as np
import datetime as dt



gc = pygsheets.authorize(service_file='client_secret.json')


year = int(input('Enter a year:'))
month = int(input('Enter a month:'))
day = int(input('Enter a day:'))



last_week_day = datetime(year, month, day)

print(last_week_day)

today = date.today()




next_day = last_week_day + timedelta(days = 1)

print(next_day)

#recovery

car_master_sheet_mumbai_vinay = gc.open("Car Master Sheet 60.40 (Vinay Bhadra)")
cms_mumbai_vinay_recovery_tab = car_master_sheet_mumbai_vinay.worksheet_by_title("Recovery")
cms_mumbai_vinay_recovery_data = pd.DataFrame(cms_mumbai_vinay_recovery_tab.get_all_records())
cms_mumbai_vinay_recovery_data = cms_mumbai_vinay_recovery_data[['Date','ETM ID','Amount']]
cms_mumbai_vinay_recovery_data.rename(columns={"ETM ID":"ETM"},inplace = True)
cms_mumbai_vinay_recovery_data['Type'] = 'RECOVERY'
cms_mumbai_vinay_recovery_data['Remarks'] = 'Vinay bhai'
cms_mumbai_vinay_recovery_data = cms_mumbai_vinay_recovery_data[['Date','ETM','Amount','Remarks','Type']]
print(cms_mumbai_vinay_recovery_data.head())

#penalty

cms_mumbai_vinay_penalty_tab = car_master_sheet_mumbai_vinay.worksheet_by_title("Penalty")
cms_mumbai_vinay_penalty_data = pd.DataFrame(cms_mumbai_vinay_penalty_tab.get_all_records())
cms_mumbai_vinay_penalty_data = cms_mumbai_vinay_penalty_data[['Date in Payout','ETM','Penalty','Fine Amount']]
cms_mumbai_vinay_penalty_data.rename(columns={"Date in Payout":"Date",'Penalty':"Remarks","Fine Amount":"Amount"},inplace = True)
cms_mumbai_vinay_penalty_data['Type'] = np.where((cms_mumbai_vinay_penalty_data['Remarks'] == "Without Intimation"), "UNSCHEDULED_LEAVES", "ACCIDENT")
print(cms_mumbai_vinay_penalty_data.head())

#b2b tab

# cms_mumbai_B2B_tab = car_master_sheet_mumbai.worksheet_by_title("B2B")
# cms_mumbai_B2B_data = pd.DataFrame(cms_mumbai_B2B_tab.get_all_records())
# cms_mumbai_B2B_data = cms_mumbai_B2B_data[['Date','ETMID','Duty','TotalDutyPayout','Fuel','Toll']]
# cms_mumbai_B2B_data.rename(columns={"ETMID":"ETM",'Duty':"Remarks"},inplace = True)

# cms_mumbai_B2B_duty = cms_mumbai_B2B_data[['Date','ETM','Remarks','TotalDutyPayout']]
# cms_mumbai_B2B_duty['Type'] = 'B2B_DUTY'
# cms_mumbai_B2B_duty.rename(columns={"TotalDutyPayout":"Amount"},inplace = True)

# cms_mumbai_B2B_fuel = cms_mumbai_B2B_data[['Date','ETM','Remarks','Fuel']]
# cms_mumbai_B2B_fuel['Type'] = 'B2B_FUEL'
# cms_mumbai_B2B_fuel.rename(columns={"Fuel":"Amount"},inplace = True)


# cms_mumbai_B2B_toll = cms_mumbai_B2B_data[['Date','ETM','Remarks','Toll']]
# cms_mumbai_B2B_toll['Type'] = 'B2B_TOLL'
# cms_mumbai_B2B_toll.rename(columns={"Toll":"Amount"},inplace = True)

#rto tab

cms_mumbai_vinay_rto_tab = car_master_sheet_mumbai_vinay.worksheet_by_title("RTO")
cms_mumbai_vinay_rto_data = pd.DataFrame(cms_mumbai_vinay_rto_tab.get_all_records())
cms_mumbai_vinay_rto_data = cms_mumbai_vinay_rto_data[['Date in Payout','ETM','Penalty','Fine Amount']]
cms_mumbai_vinay_rto_data.rename(columns={"Date in Payout":"Date",'Penalty':"Remarks","Fine Amount":"Amount"},inplace = True)
cms_mumbai_vinay_rto_data['Type'] = 'RTO_FINE'
print(cms_mumbai_vinay_rto_data.head())

#adjustment

cms_mumbai_vinay_adjustment_tab = car_master_sheet_mumbai_vinay.worksheet_by_title("Adjustments")
cms_mumbai_vinay_data = pd.DataFrame(cms_mumbai_vinay_adjustment_tab.get_all_records())
cms_mumbai_vinay_adjustment_data = cms_mumbai_vinay_data[['Date','ETM ID','Type','Remark','Amount']]
cms_mumbai_vinay_adjustment_data.rename(columns={"ETM ID":"ETM",'Remark':"Remarks","Type":"Test Type"},inplace = True)

cms_mumbai_vinay_adjustment_data['Amount'] = cms_mumbai_vinay_adjustment_data['Amount'].replace(',','', regex=True)
cms_mumbai_vinay_adjustment_data['Amount'] = cms_mumbai_vinay_adjustment_data['Amount'].replace(' ','', regex=True)

cms_mumbai_vinay_adjustment_data[['Amount']] = cms_mumbai_vinay_adjustment_data[['Amount']].apply(pd.to_numeric)

col         = 'Test Type'
col1 = 'Amount'
conditions  = [ cms_mumbai_vinay_adjustment_data[col].str.contains("fuel",case = False),cms_mumbai_vinay_adjustment_data[col].str.contains("refer",case = False), cms_mumbai_vinay_adjustment_data[col].str.contains("life",case = False), cms_mumbai_vinay_adjustment_data[col].str.contains("repair",case = False),cms_mumbai_vinay_adjustment_data[col].str.contains("pun",case = False),cms_mumbai_vinay_adjustment_data[col].str.contains("toll",case = False),cms_mumbai_vinay_adjustment_data[col].str.contains("Penalty",case = False),cms_mumbai_vinay_adjustment_data[col].str.contains("reversal",case = False),cms_mumbai_vinay_adjustment_data[col].str.contains("joining fee",case = False),cms_mumbai_vinay_adjustment_data[col1] >= 0,cms_mumbai_vinay_adjustment_data[col1] < 0 ]
choices     = [ "FUEL_ADJUSTMENT","DRIVER_REFERENCE", 'LIFETIME_INCENTIVE', 'REPAIRS','REPAIRS','TOLL','PENALTY_REVERSAL','PENALTY_REVERSAL','JOINING_FEE','OTHER_ADDITIONS','OTHER_DEDUCTIONS']
    
cms_mumbai_vinay_adjustment_data["Type"] = np.select(conditions, choices, default=np.nan)

cms_mumbai_vinay_adjustment_data = cms_mumbai_vinay_adjustment_data[['Date','ETM','Remarks','Amount','Type']]
cms_mumbai_vinay_adjustment_data.head()

#amount paid

cms_mumbai_vinay_amount_paid_tab = car_master_sheet_mumbai_vinay.worksheet_by_title("Amount Paid")
cms_mumbai_vinay_amount_paid_data = pd.DataFrame(cms_mumbai_vinay_amount_paid_tab.get_all_records())
cms_mumbai_vinay_amount_paid_data = cms_mumbai_vinay_amount_paid_data[['Date','ETM','Remarks','Amount']]
cms_mumbai_vinay_amount_paid_data['Type'] = 'BANK_TRANSFER'
print(cms_mumbai_vinay_amount_paid_data.head())

#concating 

final_mumbai_vinay_data = pd.concat([cms_mumbai_vinay_recovery_data, cms_mumbai_vinay_amount_paid_data,cms_mumbai_vinay_adjustment_data,cms_mumbai_vinay_rto_data,cms_mumbai_vinay_penalty_data],ignore_index = True)
final_mumbai_vinay_data['Date'] = pd.to_datetime(final_mumbai_vinay_data['Date'],dayfirst=True)
final_mumbai_vinay_data['start date'] = last_week_day
final_mumbai_vinay_data['end date'] = next_day
final_mumbai_vinay_data = final_mumbai_vinay_data.loc[(final_mumbai_vinay_data['start date'] <= final_mumbai_vinay_data['Date']) & (final_mumbai_vinay_data['end date'] > final_mumbai_vinay_data['Date'])]
final_mumbai_vinay_data['ETM_NEW'] = final_mumbai_vinay_data['ETM'].str.upper() 
print(final_mumbai_vinay_data.head())


final_mumbai_vinay_data = final_mumbai_vinay_data[['ETM_NEW','Amount','Type','Remarks','Date']]
final_mumbai_vinay_data.rename(columns={"ETM_NEW":"ETM"},inplace = True)
final_mumbai_vinay_data_first =  final_mumbai_vinay_data.groupby(['ETM','Type'],as_index = False).first()
final_mumbai_vinay_data_first.drop(['Amount'],axis = 1, inplace = True)
final_mumbai_vinay_data = final_mumbai_vinay_data[['ETM','Amount','Type']]
final_mumbai_vinay_data['Amount'] = final_mumbai_vinay_data['Amount'].replace(',','', regex=True)
final_mumbai_vinay_data['Amount'] = final_mumbai_vinay_data['Amount'].replace('-','0', regex=True)

final_mumbai_vinay_data[['Amount']] = final_mumbai_vinay_data[['Amount']].apply(pd.to_numeric)
print(final_mumbai_vinay_data.head())

final_mumbai_vinay_data_sum =  final_mumbai_vinay_data.groupby(['ETM','Type'],as_index = False).sum()
final_mumbai_vinay_data= final_mumbai_vinay_data_sum.merge(final_mumbai_vinay_data_first, on=["ETM","Type"])
final_mumbai_vinay_data = final_mumbai_vinay_data.loc[final_mumbai_vinay_data['Amount'] != 0]
final_mumbai_vinay_data = final_mumbai_vinay_data[['Date','ETM','Type','Amount','Remarks']]
print(final_mumbai_vinay_data.head())

#pushing the final result

cms_test = gc.open("testing")
final_result = cms_test.worksheet_by_title("cms_vinay")
final_result.set_dataframe(final_mumbai_vinay_data,'A1')


Enter a year:2022
Enter a month:07
Enter a day:01
2022-07-01 00:00:00
2022-07-02 00:00:00
         Date       ETM Amount     Remarks      Type
0  27-01-2021  ETM07248    760  Vinay bhai  RECOVERY
1  01-02-2021  ETM07576    502  Vinay bhai  RECOVERY
2  01-02-2021  ETM07907   2000  Vinay bhai  RECOVERY
3  02-02-2021  ETM07711   3700  Vinay bhai  RECOVERY
4  02-02-2021  ETM07888   1933  Vinay bhai  RECOVERY
         Date       ETM           Remarks Amount      Type
0  12-12-2021  ETM10367  Accident Penalty  -2000  ACCIDENT
1  31-12-2021  ETM10971           Penalty  -2000  ACCIDENT
2  31-12-2021  ETM11167           Penalty  -2449  ACCIDENT
3  31-12-2021  ETM07937           Penalty  -1204  ACCIDENT
4  31-12-2021  ETM11743           Penalty   -447  ACCIDENT
         Date       ETM                                   Remarks Amount  \
0  01-01-2022  ETM13520  Without Prescribed Uniform (Tourist Cab)   -500   
1  01-01-2022  ETM08761                            Jumping signal   -500   
2  01-01-2

C:\Users\sagar\anaconda3\lib\site-packages\pandas\core\frame.py:5039: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(
C:\Users\sagar\AppData\Local\Temp/ipykernel_6728/2077025393.py:94: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cms_mumbai_vinay_adjustment_data['Amount'] = cms_mumbai_vinay_adjustment_data['Amount'].replace(',','', regex=True)
C:\Users\sagar\AppData\Local\Temp/ipykernel_6728/2077025393.py:95: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = 

         Date       ETM                     Remarks    Amount           Type
0  09/10/2021  ETM06741  Positive & Deposit Refunds     453.0  BANK_TRANSFER
1  03/08/2021  ETM10848                    positive     615.0  BANK_TRANSFER
2  09/10/2021  ETM11845  Positive & Deposit Refunds  1,557.00  BANK_TRANSFER
3  09/10/2021  ETM10706        Deposit Refunds(FNF)         -  BANK_TRANSFER
4  09/10/2021  ETM10823        Deposit Refunds(FNF)         -  BANK_TRANSFER
           Date       ETM Amount     Remarks      Type start date   end date  \
2251 2022-07-01  ETM03398   2000  Vinay bhai  RECOVERY 2022-07-01 2022-07-02   
2252 2022-07-01  ETM03398    432  Vinay bhai  RECOVERY 2022-07-01 2022-07-02   
2253 2022-07-01  ETM13312   1100  Vinay bhai  RECOVERY 2022-07-01 2022-07-02   
2254 2022-07-01  ETM13312    896  Vinay bhai  RECOVERY 2022-07-01 2022-07-02   
2255 2022-07-01  ETM14067   3000  Vinay bhai  RECOVERY 2022-07-01 2022-07-02   

       ETM_NEW  
2251  ETM03398  
2252  ETM03398  
2253  

# hyderabad

In [2]:
#importing libraries

from hashlib import new
from tokenize import blank_re
import pygsheets #Importing python in google sheets
import pandas as pd #importing pandas
from pandas import Series,DataFrame
import datetime
from datetime import datetime, timedelta,date
import numpy as np
import datetime as dt



gc = pygsheets.authorize(service_file='client_secret.json')


year = int(input('Enter a year:'))
month = int(input('Enter a month:'))
day = int(input('Enter a day:'))



last_week_day = datetime(year, month, day)

print(last_week_day)

today = date.today()




next_day = last_week_day + timedelta(days = 1)

print(next_day)


#recovery
car_master_sheet_hyderabad = gc.open("Hyderabad Car Master Sheet")
cms_hyderabad_recovery_tab = car_master_sheet_hyderabad.worksheet_by_title("Recovery")
cms_hyderabad_recovery_data = pd.DataFrame(cms_hyderabad_recovery_tab.get_all_records())
cms_hyderabad_recovery_data = cms_hyderabad_recovery_data[['Payment Date','Pilot ETH ID','Amount','DM Name']]
cms_hyderabad_recovery_data.rename(columns={"Payment Date":"Date","Pilot ETH ID":"ETH",'DM Name':"Remarks"},inplace = True)
cms_hyderabad_recovery_data['Type'] = 'RECOVERY'
cms_hyderabad_recovery_data = cms_hyderabad_recovery_data[['Date','ETH','Amount','Remarks','Type']]

#penalty
cms_hyderabad_penalty_tab = car_master_sheet_hyderabad.worksheet_by_title("Penalty")
cms_hyderabad_penalty_data = pd.DataFrame(cms_hyderabad_penalty_tab.get_all_records())
cms_hyderabad_penalty_data = cms_hyderabad_penalty_data[['Date in Payout','ETM','Penalty','Fine Amount']]
cms_hyderabad_penalty_data.rename(columns={"Date in Payout":"Date","ETM":"ETH",'Penalty':"Remarks","Fine Amount":"Amount"},inplace = True)
cms_hyderabad_penalty_data['Type'] = np.where((cms_hyderabad_penalty_data['Remarks'] == "Without Intimation"), "UNSCHEDULED_LEAVES", "ACCIDENT")

#b2b tab

# cms_mumbai_B2B_tab = car_master_sheet_mumbai.worksheet_by_title("B2B")
# cms_mumbai_B2B_data = pd.DataFrame(cms_mumbai_B2B_tab.get_all_records())
# cms_mumbai_B2B_data = cms_mumbai_B2B_data[['Date','ETMID','Duty','TotalDutyPayout','Fuel','Toll']]
# cms_mumbai_B2B_data.rename(columns={"ETMID":"ETM",'Duty':"Remarks"},inplace = True)

# cms_mumbai_B2B_duty = cms_mumbai_B2B_data[['Date','ETM','Remarks','TotalDutyPayout']]
# cms_mumbai_B2B_duty['Type'] = 'B2B_DUTY'
# cms_mumbai_B2B_duty.rename(columns={"TotalDutyPayout":"Amount"},inplace = True)

# cms_mumbai_B2B_fuel = cms_mumbai_B2B_data[['Date','ETM','Remarks','Fuel']]
# cms_mumbai_B2B_fuel['Type'] = 'B2B_FUEL'
# cms_mumbai_B2B_fuel.rename(columns={"Fuel":"Amount"},inplace = True)


# cms_mumbai_B2B_toll = cms_mumbai_B2B_data[['Date','ETM','Remarks','Toll']]
# cms_mumbai_B2B_toll['Type'] = 'B2B_TOLL'
# cms_mumbai_B2B_toll.rename(columns={"Toll":"Amount"},inplace = True)

#rto tab
cms_hyderabad_rto_tab = car_master_sheet_hyderabad.worksheet_by_title("RTO")
cms_hyderabad_rto_data = pd.DataFrame(cms_hyderabad_rto_tab.get_all_records())
cms_hyderabad_rto_data = cms_hyderabad_rto_data[['Date in Payout','ETM No','Penalty','Fine Amount']]
cms_hyderabad_rto_data.rename(columns={"Date in Payout":"Date",'ETM No':'ETH','Penalty':"Remarks","Fine Amount":"Amount"},inplace = True)
cms_hyderabad_rto_data['Type'] = 'RTO_FINE'

#adjustment
cms_hyderabad_adjustment_tab = car_master_sheet_hyderabad.worksheet_by_title("Adjustments")
cms_hyderabad_adjustment_data = pd.DataFrame(cms_hyderabad_adjustment_tab.get_all_records())
cms_hyderabad_adjustment_data = cms_hyderabad_adjustment_data[['Date','ETM','type','Remark','Amount']]
cms_hyderabad_adjustment_data.rename(columns={"ETM":"ETH",'Remark':"Remarks","type":"Test Type"},inplace = True)

cms_hyderabad_adjustment_data['Amount'] = cms_hyderabad_adjustment_data['Amount'].replace(',','', regex=True)
cms_hyderabad_adjustment_data['Amount'] = cms_hyderabad_adjustment_data['Amount'].replace(' ','', regex=True)

cms_hyderabad_adjustment_data[['Amount']] = cms_hyderabad_adjustment_data[['Amount']].apply(pd.to_numeric)

col         = 'Test Type'
col1 = 'Amount'
conditions  = [ cms_hyderabad_adjustment_data[col].str.contains("fuel",case = False),cms_hyderabad_adjustment_data[col].str.contains("refer",case = False), cms_hyderabad_adjustment_data[col].str.contains("life",case = False), cms_hyderabad_adjustment_data[col].str.contains("repair",case = False),cms_hyderabad_adjustment_data[col].str.contains("pun",case = False),cms_hyderabad_adjustment_data[col].str.contains("toll",case = False),cms_hyderabad_adjustment_data[col].str.contains("Penalty",case = False),cms_hyderabad_adjustment_data[col].str.contains("reversal",case = False),cms_hyderabad_adjustment_data[col].str.contains("joining fee",case = False),cms_hyderabad_adjustment_data[col1] >= 0,cms_hyderabad_adjustment_data[col1] < 0 ]
choices     = [ "FUEL_ADJUSTMENT","DRIVER_REFERENCE", 'LIFETIME_INCENTIVE', 'REPAIRS','REPAIRS','TOLL','PENALTY_REVERSAL','PENALTY_REVERSAL','JOINING_FEE','OTHER_ADDITIONS','OTHER_DEDUCTIONS']
    
cms_hyderabad_adjustment_data["Type"] = np.select(conditions, choices, default=np.nan)

cms_hyderabad_adjustment_data = cms_hyderabad_adjustment_data[['Date','ETH','Remarks','Amount','Type']]

#amount paid
cms_hyderabad_amount_paid_tab = car_master_sheet_hyderabad.worksheet_by_title("Amount Paid")
cms_hyderabad_amount_paid_data = pd.DataFrame(cms_hyderabad_amount_paid_tab.get_all_records())
cms_hyderabad_amount_paid_data = cms_hyderabad_amount_paid_data[['Date','ETH','Remarks','Amount']]
cms_hyderabad_amount_paid_data['Type'] = 'BANK_TRANSFER'

#concating 

final_hyderabad_data = pd.concat([cms_hyderabad_recovery_data, cms_hyderabad_amount_paid_data,cms_hyderabad_adjustment_data,cms_hyderabad_rto_data,cms_hyderabad_penalty_data],ignore_index = True)
final_hyderabad_data['Date'] = pd.to_datetime(final_hyderabad_data['Date'],dayfirst=True)
final_hyderabad_data['start date'] = last_week_day
final_hyderabad_data['end date'] = next_day
final_hyderabad_data = final_hyderabad_data.loc[(final_hyderabad_data['start date'] <= final_hyderabad_data['Date']) & (final_hyderabad_data['end date'] > final_hyderabad_data['Date'])]
final_hyderabad_data['ETH_NEW'] = final_hyderabad_data['ETH'].str.upper() 

final_hyderabad_data = final_hyderabad_data[['ETH_NEW','Amount','Type','Remarks','Date']]
final_hyderabad_data.rename(columns={"ETH_NEW":"ETH"},inplace = True)

final_hyderabad_data_first =  final_hyderabad_data.groupby(['ETH','Type'],as_index = False).first()

final_hyderabad_data_first.drop(['Amount'],axis = 1, inplace = True)
final_hyderabad_data = final_hyderabad_data[['ETH','Amount','Type']]
final_hyderabad_data['Amount'] = final_hyderabad_data['Amount'].replace(',','', regex=True)
final_hyderabad_data[['Amount']] = final_hyderabad_data[['Amount']].apply(pd.to_numeric)

final_hyderabad_data_sum =  final_hyderabad_data.groupby(['ETH','Type'],as_index = False).sum()
final_hyderabad_data= final_hyderabad_data_sum.merge(final_hyderabad_data_first, on=["ETH","Type"])
final_hyderabad_data = final_hyderabad_data.loc[final_hyderabad_data['Amount'] != 0]
final_hyderabad_data = final_hyderabad_data[['Date','ETH','Type','Amount','Remarks']]
final_hyderabad_data

#pushing the final result

cms_test = gc.open("testing")
final_result = cms_test.worksheet_by_title("cms_hyd")
final_result.set_dataframe(final_hyderabad_data,'A1')


Enter a year:2022
Enter a month:07
Enter a day:01
2022-07-01 00:00:00
2022-07-02 00:00:00
